In [1]:
#importing all the necessary libraries
import requests
from pytrends.request import TrendReq
from pytrends import dailydata
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
pytrends = TrendReq(hl='en-US', tz=360)

In [3]:
companies = ['netflix', 'nike', 'starbucks', 'yelp', 'tesla', 'amazon']

In [4]:
#function which return the google trends per hour from a keyword, given the keyword and begin/end dates
def get_hourly_interest(company, begin_year, begin_month_start, begin_day, end_year, end_month, end_day):
    hourly = pytrends.get_historical_interest(company, year_start = begin_year, month_start=begin_month, day_start = begin_day, hour_start = 0, year_end = end_year, month_end=end_month, day_end=end_day, hour_end = 0, sleep = 5)
    hourly.to_csv(f'../02 CSV_files/trends/{company}_trends_from{begin_day}{begin_month}{begin_year}_to{end_day}{end_month}{end_year}.csv')

In [5]:
#function which returns the interest in a keyword on a weekly basis for the last 5 years. It saves it as the keyword weekly trends
def download_and_save_interest(company_name):
    pytrends.build_payload(company_name)
    trends = pytrends.interest_over_time()
    trends.to_csv(f'../02 CSV_files/trends/{company_name[0]}_weekly_trends.csv')

In [6]:
#function which reads a csv file, given the path
def read_and_create_df(path):
    return pd.read_csv(path)

In [7]:
#function which cleans the trends dataframe returned by the function get_hourly_interest or download_and_save_interest
def cleaning_trends_dataframe(df):
    df['hour'] = df['date'].apply(pd.to_datetime)
    df['date'] = df['hour'].dt.date
    df['week'] = df['hour'].dt.week
    df['month'] = df['hour'].dt.month
    df['year'] = df['hour'].dt.year
    df.drop('isPartial', axis = 1, inplace = True)
    return df

In [8]:
#function which adds 3 features to the trends dataframes (last weeks trends, % interest and the name of the company)
def add_features_df(df):
    df['last_week'] = df[df.columns[1]].shift()
    df['perc_incr'] = (df[df.columns[1]] - df['last_week']) / df['last_week']
    company_name = df.columns[1]
    df['company'] = company_name
    return df

In [9]:
#function which returns the 20 weeks with higher increase percentage in the trends and 
#gets the hourly interest of the previous week and the mentioned week. Appending all the gathered date into one dataframe
def hourly_peaks_interest(weekly_df):
    total_peaks = pd.DataFrame()
    for date in weekly_df.sort_values(by = 'perc_incr', ascending = False).head(20).date:
        begin_date = date - dt.timedelta(days = 7)
        end_date = date + dt.timedelta(days = 7)
        hourly = pytrends.get_historical_interest([weekly_df.columns[1]], year_start = begin_date.year, month_start=begin_date.month, day_start = begin_date.day, hour_start = 0, year_end = end_date.year, month_end=end_date.month, day_end=end_date.day, hour_end = 0, sleep = 5)
        company_name = weekly_df.columns[1]
        weekly_df['company'] = company_name
        total_peaks = total_peaks.append(hourly)
    return total_peaks

In [10]:
def all_hourly_trends_dataframes(companys_list):
    final_df = pd.DataFrame()
    for company in companies:
        download_and_save_interest([company])
        company_df = read_and_create_df(f'../02 CSV_files/trends/{company}_weekly_trends.csv')
        cleaned_df = cleaning_trends_dataframe(company_df)
        complete_df = add_features_df(cleaned_df)
        peaks_df = hourly_peaks_interest(complete_df)
        final_df = final_df.append(peaks_df)
        return final_df

In [11]:
def hourly_trends_dataframe(company):
    download_and_save_interest(company)
    company_df = read_and_create_df(f'../02 CSV_files/trends/{company}_weekly_trends.csv')
    cleaned_df = cleaning_trends_dataframe(company_df)
    complete_df = add_features_df(cleaned_df)
    peaks_df = hourly_peaks_interest(complete_df)
    final_df = final_df.append(peaks_df)
    return final_df

#final_df = all_hourly_trends_dataframes(companies)
#final_df

In [ ]:
df = pd.read_csv('../02 CSV_files/hourly_peaks.csv')

In [12]:
#function which collects the daily google trends of a keyword from the specified timerange
def get_daily_trends(company, year_begin, month_begin, year_end, month_end):
    df = dailydata.get_daily_data(company, year_begin, month_begin, year_end, month_end)
    df.drop(columns = ['isPartial',f'{company}_monthly', 'scale'], inplace = True)
    df = df.reset_index()
    df.date = pd.to_datetime(df.date)
    df.rename(columns = {f'{company}_unscaled':'unscaled', f'{company}':'scaled'}, inplace = True)
    df['name'] = company
    return df

In [13]:
#function which iterates through a list of keywords and saves it in the trends file
def companies_daily_trends(companies_list, begin_year,begin_month, end_year, end_month):
    daily_trends = pd.DataFrame(columns = ['date', 'unscaled','scaled'])
    for company in companies_list:
        for year in range(begin_year,end_year+1):
            new_df = get_daily_trends(company,year,begin_month,year,end_month)
            daily_trends = daily_trends.append(new_df)
            daily_trends.to_csv(f'../02 CSV_files/trends/daily_trends_{company}{year}.csv')

In [16]:
#function which iterates through a list of keywords and saves it in the trends file
def companies_daily_trends_2(companies_list, begin_year,begin_month, end_year, end_month):
    daily_trends = pd.DataFrame(columns = ['date', 'unscaled','scaled'])
    for company in companies_list:
        new_df = get_daily_trends(company,begin_year,begin_month,end_year,end_month)
        daily_trends = daily_trends.append(new_df)
        daily_trends.to_csv(f'../02 CSV_files/trends/daily_trends_{company}.csv')

In [17]:
companies_daily_trends(companies, 2015,1, 2019,12)
companies_daily_trends(companies, 2020,1, 2020,5)
companies_daily_trends_2(companies, 2015,1, 2019,12)
companies_daily_trends_2(companies, 2020,1, 2020,5)

netflix:2015-01-01 2015-01-31
netflix:2015-02-01 2015-02-28
netflix:2015-03-01 2015-03-31
netflix:2015-04-01 2015-04-30
netflix:2015-05-01 2015-05-31
netflix:2015-06-01 2015-06-30
netflix:2015-07-01 2015-07-31
netflix:2015-08-01 2015-08-31
netflix:2015-09-01 2015-09-30
netflix:2015-10-01 2015-10-31
netflix:2015-11-01 2015-11-30
netflix:2015-12-01 2015-12-31
netflix:2016-01-01 2016-01-31
netflix:2016-02-01 2016-02-29
netflix:2016-03-01 2016-03-31
netflix:2016-04-01 2016-04-30
netflix:2016-05-01 2016-05-31
netflix:2016-06-01 2016-06-30
netflix:2016-07-01 2016-07-31
netflix:2016-08-01 2016-08-31
netflix:2016-09-01 2016-09-30
netflix:2016-10-01 2016-10-31
netflix:2016-11-01 2016-11-30
netflix:2016-12-01 2016-12-31
netflix:2017-01-01 2017-01-31
netflix:2017-02-01 2017-02-28
netflix:2017-03-01 2017-03-31
netflix:2017-04-01 2017-04-30
netflix:2017-05-01 2017-05-31
netflix:2017-06-01 2017-06-30
netflix:2017-07-01 2017-07-31
netflix:2017-08-01 2017-08-31
netflix:2017-09-01 2017-09-30
netflix:20

C:\Users\usuario\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


nike:2015-01-01 2015-01-31
nike:2015-02-01 2015-02-28
nike:2015-03-01 2015-03-31
nike:2015-04-01 2015-04-30
nike:2015-05-01 2015-05-31
nike:2015-06-01 2015-06-30
nike:2015-07-01 2015-07-31
nike:2015-08-01 2015-08-31
nike:2015-09-01 2015-09-30
nike:2015-10-01 2015-10-31
nike:2015-11-01 2015-11-30
nike:2015-12-01 2015-12-31
nike:2016-01-01 2016-01-31
nike:2016-02-01 2016-02-29
nike:2016-03-01 2016-03-31
nike:2016-04-01 2016-04-30
nike:2016-05-01 2016-05-31
nike:2016-06-01 2016-06-30
nike:2016-07-01 2016-07-31
nike:2016-08-01 2016-08-31
nike:2016-09-01 2016-09-30
nike:2016-10-01 2016-10-31
nike:2016-11-01 2016-11-30
nike:2016-12-01 2016-12-31
nike:2017-01-01 2017-01-31
nike:2017-02-01 2017-02-28
nike:2017-03-01 2017-03-31
nike:2017-04-01 2017-04-30
nike:2017-05-01 2017-05-31
nike:2017-06-01 2017-06-30
nike:2017-07-01 2017-07-31
nike:2017-08-01 2017-08-31
nike:2017-09-01 2017-09-30
nike:2017-10-01 2017-10-31
nike:2017-11-01 2017-11-30
nike:2017-12-01 2017-12-31
nike:2018-01-01 2018-01-31
n

amazon:2018-12-01 2018-12-31
amazon:2019-01-01 2019-01-31
amazon:2019-02-01 2019-02-28
amazon:2019-03-01 2019-03-31
amazon:2019-04-01 2019-04-30
amazon:2019-05-01 2019-05-31
amazon:2019-06-01 2019-06-30
amazon:2019-07-01 2019-07-31
amazon:2019-08-01 2019-08-31
amazon:2019-09-01 2019-09-30
amazon:2019-10-01 2019-10-31
amazon:2019-11-01 2019-11-30
amazon:2019-12-01 2019-12-31


In [ ]:
#reading all the saved csv in the trends file, given a list of keywords and years
def reading_trends_df(companies_list, begin_year, end_year):
    trends_df = pd.DataFrame()
    for company in companies_list:
        for year in range(begin_year,end_year+1):
            new_company = pd.read_csv(f'../02 CSV_files/trends/daily_trends_{company}{year}.csv', index_col = 0)
            trends_df = trends_df.append(new_company)
    return trends_df

trends_df = reading_trends_df(companies,2015,2020)

In [18]:
#reading all the saved csv in the trends file, given a list of keywords and years
def reading_trends_df_2(companies_list):
    trends_df = pd.DataFrame()
    for company in companies_list:
        new_company = pd.read_csv(f'../02 CSV_files/trends/daily_trends_{company}.csv', index_col = 0)
        trends_df = trends_df.append(new_company)
    return trends_df

trends_df_2 = reading_trends_df_2(companies)

In [20]:
#cleaning the trends dataframe before shifting the columns
def cleaning_trends_finale(df):
    df['date'] = pd.to_datetime(df.date)
    df = df.groupby(['date','name']).agg('max').reset_index()
    df = df.sort_values(by=['name','date'])
    df.drop(columns = 'unscaled', inplace = True)
    df.dropna(axis = 0, inplace = True)
    return df

In [21]:
trends_df_2 = cleaning_trends_finale(trends_df_2)

In [23]:
#creating columns referencing previous rows, in order to see the evolution of the trends
def shift_n_rows(df,col_name,n_rows):
    for col in range(1,n_rows+1):
        df[f'{col_name}_day_{col}'] = df[col_name].shift(col)
    return df

In [24]:
#applying the shift function individualy to each company and combining it afterwards into one dataframe
def create_shifted_df(companies_list, df ,col_name, n_of_shifts):
    shift = {}
    shifted = pd.DataFrame()
    for company in companies:
        new_df = df[df['name'] == company]
        new_df = shift_n_rows(new_df, col_name, n_of_shifts)
        shift[company] = new_df
        shifted = shifted.append(new_df)
    shifted.dropna(axis = 0, inplace = True)
    shifted.date = pd.to_datetime(shifted.date)
    shifted.to_csv('../02 CSV_files/csv_finals/final_trends.csv')
    return shifted

In [ ]:
trends_shifted = create_shifted_df(companies, trends_df, 'scaled', 14)

In [ ]:
trends_shifted.to_csv('../02 CSV_files/csv_finals/final_trends.csv')

In [ ]:
trends_shifted

In [25]:
trends_shifted_2 = create_shifted_df(companies, trends_df_2, 'scaled', 14)
trends_shifted_2.to_csv('../02 CSV_files/csv_finals/final_trends_2.csv')

C:\Users\usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
trends_shifted_2

,date,name,scaled,scaled_day_1,scaled_day_2,scaled_day_3,scaled_day_4,scaled_day_5,scaled_day_6,scaled_day_7,scaled_day_8,scaled_day_9,scaled_day_10,scaled_day_11,scaled_day_12,scaled_day_13,scaled_day_14
103,2015-01-18,netflix,43.07,42.34,30.74,28.42,27.84,26.68,31.32,42.92,46.08,35.20,32.64,34.56,34.56,39.04,55.04
109,2015-01-19,netflix,37.17,43.07,42.34,30.74,28.42,27.84,26.68,31.32,42.92,46.08,35.20,32.64,34.56,34.56,39.04
115,2015-01-20,netflix,29.50,37.17,43.07,42.34,30.74,28.42,27.84,26.68,31.32,42.92,46.08,35.20,32.64,34.56,34.56
121,2015-01-21,netflix,28.32,29.50,37.17,43.07,42.34,30.74,28.42,27.84,26.68,31.32,42.92,46.08,35.20,32.64,34.56
127,2015-01-22,netflix,28.32,28.32,29.50,37.17,43.07,42.34,30.74,28.42,27.84,26.68,31.32,42.92,46.08,35.20,32.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10926,2019-12-27,amazon,55.25,59.50,54.40,46.75,52.70,57.80,69.84,67.90,68.87,71.78,72.75,73.72,75.66,73.92,68.16
10932,2019-12-28,amazon,52.70,55.25,59.50,54.40,46.75,52.70,57.80,69.84,67.90,68.87,71.78,72.75,73.72,75.66,73.92
10938,2019-12-29,amazon,44.25,52.70,55.25,59.50,54.40,46.75,52.70,57.80,69.84,67.90,68.87,71.78,72.75,73.72,75.66
10944,2019-12-30,amazon,43.50,44.25,52.70,55.25,59.50,54.40,46.75,52.70,57.80,69.84,67.90,68.87,71.78,72.75,73.72
